<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Анализ" data-toc-modified-id="Анализ-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Анализ</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Тестирование" data-toc-modified-id="Тестирование-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Тестирование</a></span></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Вывод</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

#  Прогнозирование заказов такси

Компания «Чётенькое такси» собрала исторические данные о заказах такси в аэропортах. Чтобы привлекать больше водителей в период пиковой нагрузки, нужно спрогнозировать количество заказов такси на следующий час. Постройте модель для такого предсказания.

Значение метрики *RMSE* на тестовой выборке должно быть не больше 48.

Вам нужно:

1. Загрузить данные и выполнить их ресемплирование по одному часу.
2. Проанализировать данные.
3. Обучить разные модели с различными гиперпараметрами. Сделать тестовую выборку размером 10% от исходных данных.
4. Проверить данные на тестовой выборке и сделать выводы.


Данные лежат в файле `taxi.csv`. Количество заказов находится в столбце `num_orders` (от англ. *number of orders*, «число заказов»).

## Подготовка

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time

from statsmodels.tsa.seasonal import seasonal_decompose

from sklearn.model_selection import train_test_split, cross_val_score, learning_curve, validation_curve
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_validate as cval
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.dummy import DummyRegressor

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor

from xgboost import XGBRegressor
from catboost import CatBoostRegressor, Pool, cv
from lightgbm import LGBMRegressor

In [ ]:
import warnings
warnings.filterwarnings('ignore')
rs = 12345

In [ ]:
pd.set_option('display.max_columns', 90)
pd.set_option('display.max_rows', 90)
plt.rcParams['axes.titlesize'] = 15
plt.rcParams['axes.titleweight'] = 2
pd.set_option("display.precision", 2)

Посмотрю датасет

In [ ]:
taxi = pd.read_csv('/datasets/taxi.csv', parse_dates=[0], index_col=[0])
taxi.info()

In [ ]:
taxi.head()

Надо убедиться, что данные являются временным рядом. Для этого рассмотрю график и детальнее разберу дату и время, проанализирую их и синтезирую их обратно (удалю лишнее).

In [ ]:
taxi.plot()

In [ ]:
taxi['year'] = taxi.index.year
taxi['month'] = taxi.index.month
taxi['day'] = taxi.index.day
taxi['hour'] = taxi.index.hour
taxi['minute'] = taxi.index.minute
taxi['second'] = taxi.index.second

In [ ]:
taxi['year'].unique()

In [ ]:
taxi['month'].unique()

In [ ]:
taxi['day'].unique()

In [ ]:
taxi['hour'].unique()

In [ ]:
taxi['minute'].unique()

In [ ]:
taxi['second'].unique()

Уникальные значения дат и времени нормальны, с ними все хорошо

In [ ]:
taxi['num_orders'].unique()

In [ ]:
taxi['date'] = pd.to_datetime(taxi.index)
print(taxi['date'].diff().max() - taxi['date'].diff().min())
taxi.drop('date', axis=1, inplace=True)

разрывов нет, шаг ряда одинаков и поэтому ряд является временным. Удалю созданные признаки и произведу resample с 10 минут до 1 часа

In [ ]:
taxi.drop(['year', 'month', 'day', 'hour', 'minute', 'second'], axis=1, inplace=True)

In [ ]:
taxi = taxi.resample('1H').sum()
taxi

Данные готовы для анализа, осталось только на всякий случай произвести сортировку временного ряда по индексу

In [ ]:
taxi.sort_index(inplace=True)

## Анализ

In [ ]:
taxi['num_orders'].plot();

In [ ]:
taxi_decomposed = seasonal_decompose(taxi.resample('1D').mean())

fig = plt.figure(figsize=(14, 6), facecolor='gray')
fig.suptitle('Временной ряд', fontsize=16)
plt.subplots_adjust(wspace=0.2, hspace=0.3)

ax = fig.add_subplot(1, 3, 1)
ax=plt.gca()
taxi_decomposed.trend.plot(ax=ax);
ax.set_title('Trend')
ax.grid(True)

ax = fig.add_subplot(1, 3, 2)
ax=plt.gca()
taxi_decomposed.seasonal.plot(ax=ax);
ax.set_title('Seasonality')
ax.grid(True)

ax = fig.add_subplot(1, 3, 3)
ax=plt.gca()
taxi_decomposed.resid.plot(ax=ax);
ax.set_title('Residual')
ax.grid(True)
plt.tight_layout()
plt.show()

Тренд явно имеет восходящее значение, следовательно средняя величина изменяется. Стандартное отклонение также является разным, это видно по графику тренда. В августе 2018 года вообще начался экспоненциальный рост (или около того). Это является признаком нестационарности ряда. Для чего нужны остатки пока не понятно. Сезонность надо рассмотреть поближе, при resample=1H график плохо виден.

In [ ]:
plt.figure(figsize=(18, 8))
taxi_decomposed.seasonal.plot(ax=plt.gca());
plt.title('Seasonality', fontsize=20)
plt.grid(True)
plt.tight_layout()
plt.show()

Сезонность есть, форма ее необычная - синусом или косинусом не опишешь. Длительность цикла сезонности составляет примерно 1 раз в неделю.

In [ ]:
plt.figure(figsize=(18, 8))
taxi['mean'] = taxi['num_orders'].rolling(100).mean()
taxi['std'] = taxi['num_orders'].rolling(100).std()
taxi.plot(ax=plt.gca());
plt.grid(True)
plt.show()
taxi.drop(['mean', 'std'], axis=1, inplace=True)

## Обучение

In [ ]:
def make_features(data, max_lag, rolling_mean_size):
    data['month'] = data.index.month
    data['day'] = data.index.day
    data['dayofweek'] = data.index.dayofweek
    data['hour'] = data.index.hour
    
    for lag in range(1, max_lag + 1):
        data['lag_{}'.format(lag)] = data['num_orders'].shift(lag)

    data['rolling_mean'] = data['num_orders'].shift().rolling(rolling_mean_size).mean()

In [ ]:
def search_best_params(data, lags, rolling_mean, model):
    taxi = data
    make_features(taxi, lags, rolling_mean)
    taxi.dropna(inplace=True)

    features = taxi.drop('num_orders', axis=1)
    target = taxi['num_orders']
    features_train, features_test, target_train, target_test = train_test_split(features, target, shuffle=False, 
                                                                                test_size=0.1, random_state=rs)

    scaler = StandardScaler()
    scaler.fit(features_train)
    features_train = pd.DataFrame(scaler.transform(features_train), columns=features_train.columns, index=features_train.index)
    features_test = pd.DataFrame(scaler.transform(features_test), columns=features_test.columns, index=features_test.index)

    model.fit(features_train, target_train)
    target_predict = model.predict(features_test)
    RMSE = mean_squared_error(target_test, target_predict) ** 0.5
    return(RMSE, model)

In [ ]:
def search_best_model(df, model, time_split=None):
    best_lag = 0
    best_roll = 0
    best_score = 100
    best_model = None
    for lag in range(1, 5, 1):
        for roll_mean in range(1, 5, 1):
            taxi_copy = df.copy()
            if time_split == None:
                rmse_model, model_best = search_best_params(data=taxi_copy, lags=lag, rolling_mean=roll_mean, model=model)
            elif time_split != None:
                rmse_model, model_best = search_best_params_and_hyperparams(data=taxi_copy, lags=lag, rolling_mean=roll_mean,
                                                                            model=model, time_split=time_split)
            if rmse_model < best_score:
                best_model = model_best
                best_score = rmse_model
                best_lag = lag
                best_roll = roll_mean
    print(f'best_RMSE={best_score:.2f}, best_lag={best_lag}, best_rolling_mean={best_roll}, best_model={best_model}')
    return(best_score, best_lag, best_roll, best_model)

In [ ]:
start_time = time.time()
lr_score, lr_lag, lr_roll, lr_model = search_best_model(df=taxi, model=LinearRegression())
finish_time = time.time()
lr_time = finish_time - start_time

best_RMSE=52.431388538106695, best_lag=3, best_rolling_mean=2
CPU times: user 712 ms, sys: 253 ms, total: 965 ms
Wall time: 927 ms

In [ ]:
def search_best_params_and_hyperparams(data, lags, rolling_mean, model, time_split):
    taxi = data
    make_features(taxi, lags, rolling_mean)
    taxi.dropna(inplace=True)

    features = taxi.drop('num_orders', axis=1)
    target = taxi['num_orders']
    features_train, features_test, target_train, target_test = train_test_split(features, target, shuffle=False, 
                                                                                test_size=0.1, random_state=rs)

    scaler = StandardScaler()
    scaler.fit(features_train)
    features_train = pd.DataFrame(scaler.transform(features_train), columns=features_train.columns, index=features_train.index)
    features_test = pd.DataFrame(scaler.transform(features_test), columns=features_test.columns, index=features_test.index)
 
    params = {'n_estimators': [i for i in range(100, 141, 10)],
             'learning_rate': [i for i in np.arange(0.04, 0.071, 0.01)]}

    gs = GridSearchCV(model, params, cv=time_split, n_jobs=-1)
    gs.fit(features_train, target_train)

    model.set_params(**gs.best_params_)
    model.fit(features_train, target_train)
    target_predict = model.predict(features_test)
    RMSE = mean_squared_error(target_test, target_predict) ** 0.5
    gs = None
    return(RMSE, model)

In [ ]:
tscv = TimeSeriesSplit()

In [ ]:
start_time = time.time()
gbr_score, gbr_lag, gbr_roll, gbr_model = search_best_model(df=taxi, 
                                                model=GradientBoostingRegressor(max_depth=6, random_state=rs), time_split=tscv)
finish_time = time.time()
gbr_time = finish_time - start_time

In [ ]:
start_time = time.time()
xgb_score, xgb_lag, xgb_roll, xgb_model = search_best_model(df=taxi, 
                                                model=XGBRegressor(max_depth=6, random_state=rs, verbosity=0), time_split=tscv)
finish_time = time.time()
xgb_time = finish_time - start_time

In [ ]:
start_time = time.time()
lgb_score, lgb_lag, lgb_roll, lgb_model = search_best_model(df=taxi, 
                                                model=LGBMRegressor(max_depth=6, random_state=rs), time_split=tscv)
finish_time = time.time()
lgb_time = finish_time - start_time

In [ ]:
start_time = time.time()
cat_score, cat_lag, cat_roll, cat_model = search_best_model(df=taxi, 
                                            model=CatBoostRegressor(max_depth=6, random_state=rs, verbose=0), time_split=tscv)
finish_time = time.time()
cat_time = finish_time - start_time

## Тестирование

Тестирование было произведено в предыдущем пунке внутри функции, результаты вывожу в следующем пункте

## Вывод

In [ ]:
score = [lr_score, gbr_score, xgb_score, lgb_score]
lag = [lr_lag, gbr_lag, xgb_lag, lgb_lag]
roll = [lr_roll, gbr_roll, xgb_roll, lgb_roll]
models = [LinearRegression(), GradientBoostingRegressor(), XGBRegressor(), LGBMRegressor()]
time = [lr_time, gbr_time, xgb_time, lgb_time]

compare = []
for index,content in enumerate(score):
    compare.append([score[index], lag[index], roll[index], time[index]])
summary = pd.DataFrame(compare, index=[type(i).__name__ for i in models], 
                       columns=['RMSE_score', 'max_lag', 'rolling_mean_size', 'train_time'])
summary.sort_values(by='RMSE_score', ascending=False)

Посчитал на маленьком диапазоне параметров, чтобы время сэкономить. Если не рассматривать CatBoost, то лучшей метрики я достиг в библиотеке LGBMRegressor - 41.42. Это меньше заданной величины 48. Время расчета этой модели так же минимально. 